## Dependecies

In [ ]:
# Image 
import cv2
from sklearn.preprocessing import normalize
from albumentations import Compose, ShiftScaleRotate, RandomGamma

# Machine Learning
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, TensorBoard

import tensorflow.keras.backend as K

# Helper libraries
import os
import time
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support, classification_report

## Define Hyperparameters

In [ ]:
SHAPE      = (256, 256, 1) # target image shape
BATCH_SIZE = 24   # training batch size
N_SPLITS   = 3    # k-fold splits
EPOCHS     = 100  # training epochs
SEED       = 1881 # Used for reproduction

DIR             = 'data/OCT2017/' # DATA Location Directory
MODEL_SAVE_PATH = "models/"       # K-Fold Model Path
LOGS_PATH       = "logs/"         # Tensorboard Records Path

In [ ]:
os.listdir(DIR)

## Data Generator

In [ ]:
class OCT_TrainingDataGenerator:
    
    """
    input_shape --> TUPLE.wanted image size
    batch_size  --> INT.yielding data size for every iteration
    orders      --> LIST.which images will be used. max=len(all_images). it can be used for K-fold(CV).
    DIR         --> STR.DIR directory must consists "test", "train" dirs.
    augment     --> BOOL. 
    """
    
    def __init__(self, input_shape, batch_size, orders, DIR, augment=True):
        self.SHAPE       = input_shape
        self.BATCH_SIZE  = batch_size
        self.arr         = orders
        self.aug         = augment
        
        self.DIR         = DIR
        self.SUBDIRS     = ['train']
        self.SUB_SUBDIRS = ['CNV', "DME", "DRUSEN", "NORMAL"] 
    
    def __len__(self):
        return len(self.get_img_paths())
    
    def get_img_paths(self):
        # Get all training image paths.
        arr = []
        for x in self.SUBDIRS:
            for y in self.SUB_SUBDIRS:
                for im in os.listdir(os.path.join(self.DIR,x,y)):
                    if ".jpeg" in im:
                        arr.append(os.path.join(self.DIR,x,y,im))
        return arr

    def get_img(self, img_path):
        img = cv2.imread(img_path)
        return np.array(img)
    
    def augmenting(self, img):
        if self.aug:
            augment = Compose([ShiftScaleRotate(p=0.5, shift_limit=0.01, scale_limit=0., rotate_limit=10),
                               RandomGamma(p=0.4)])  
        else:
            augment = Compose([])  

        img = augment(image=img)['image']
        return img
    
    def resize_and_normalize(self, img):
        img = cv2.resize(img, SHAPE[:2])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # OpenCV loads image as BGR, now we should convert it to grayscale.
        img = normalize(img) # normalize the image in 0-1 range for faster training.
        return np.expand_dims(img, axis=2)
    
    def data_generator(self):
        img_paths = np.array(self.get_img_paths())
        np.random.shuffle(img_paths)
        
        while True:
            x = np.empty((self.BATCH_SIZE,)+self.SHAPE, dtype=np.float16)
            y = np.empty((self.BATCH_SIZE, 4), dtype=np.float16)
            
            batch = np.random.choice(self.arr, self.BATCH_SIZE)

            for ix, id_ in enumerate(batch):
                # x
                img_path = img_paths[id_]
                img = self.get_img(img_path)
                augmented_img = self.augmenting(img)
                img = self.resize_and_normalize(augmented_img)
                  
                # y 
                if 'CNV' in img_path:
                    label = [1,0,0,0]
                elif 'DME' in img_path:
                    label = [0,1,0,0]
                elif 'DRUSEN' in img_path:
                    label = [0,0,1,0]
                elif 'NORMAL' in img_path:
                    label = [0,0,0,1]
                    
                # Store the values    
                x[ix] = img
                y[ix] = label

            yield x,y

In [ ]:
# Testing data generator library
dataset = OCT_TrainingDataGenerator(SHAPE, 1, len(range(10)), DIR=DIR, augment=True)

for ix, data in enumerate(dataset.data_generator()):
    x, y = data
    print(x)
    print(x.shape)
    print(y)
    print(y.shape)
    
    if ix==0:
        break

## Metrics

In [ ]:
# credit: https://stackoverflow.com/a/45305384

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precisionx = precision(y_true, y_pred)
    recallx = recall(y_true, y_pred)
    return 2*((precisionx*recallx)/(precisionx+recallx+K.epsilon()))

## Learning Rate Scheduler

In [ ]:
# credit: https://gist.github.com/jeremyjordan/5a222e04bb78c242f5763ad40626c452

class SGDRScheduler(Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.
    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)

## Creating Model

In [ ]:
# credit: https://github.com/titu1994/keras-squeeze-excite-network/blob/master/se.py

def squeeze_excite_block(input, ratio=16):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

In [ ]:
def create_model():
    
    dropRate = 0.3
    
    init = Input(SHAPE)
    x = Conv2D(32, (3, 3), activation=None, padding='same')(init) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3), activation=None, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x1 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(64, (3, 3), activation=None, padding='same')(x1)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = squeeze_excite_block(x)
    x = Conv2D(64, (5, 5), activation=None, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = squeeze_excite_block(x)
    x = Conv2D(64, (3, 3), activation=None, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x2 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(128, (3, 3), activation=None, padding='same')(x2)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = squeeze_excite_block(x)
    x = Conv2D(128, (2, 2), activation=None, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = squeeze_excite_block(x)
    x = Conv2D(128, (3, 3), activation=None, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x3 = MaxPooling2D((2,2))(x)
    
    ginp1 = UpSampling2D(size=(2, 2), interpolation='bilinear')(x1)
    ginp2 = UpSampling2D(size=(4, 4), interpolation='bilinear')(x2)
    ginp3 = UpSampling2D(size=(8, 8), interpolation='bilinear')(x3)
    
    concat = Concatenate()([ginp1, ginp2, ginp3]) 
    gap = GlobalAveragePooling2D()(concat)
    
    x = Dense(256, activation=None)(gap)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropRate)(x)
    
    x = Dense(256, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Dense(4, activation='softmax')(x)
    
    model = Model(init, x)
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=1e-3, momentum=0.9), metrics=['acc', precision, recall, f1])
    return model

In [ ]:
model = create_model()
model.summary()

## Train the model

In [ ]:
kf = KFold(n_splits=N_SPLITS, random_state=SEED, shuffle=True)

dataset = OCT_TrainingDataGenerator(SHAPE, BATCH_SIZE, 0, DIR, augment=False)

for ix, (train_index, test_index) in enumerate(kf.split(range(len(dataset)))):
    print("** FOLD {} **".format(ix))       
    
    tg = OCT_TrainingDataGenerator(SHAPE, BATCH_SIZE, train_index, DIR, augment=True)  # Create training data generator with augmentation is true.
    vg = OCT_TrainingDataGenerator(SHAPE, BATCH_SIZE, test_index , DIR, augment=False) # Create validation data generator with augmentation is false.
        
    schedule = SGDRScheduler(min_lr=1e-6,
                             max_lr=1e-3,
                             steps_per_epoch=np.ceil(EPOCHS/BATCH_SIZE),
                             lr_decay=0.8,
                             cycle_length=10,
                             mult_factor=1.)

    model_ckpt = os.path.join(MODEL_SAVE_PATH,"heysaw_fold_"+str(ix)+".h5")
    callbacks = [ModelCheckpoint(model_ckpt, monitor='val_loss', mode='min', verbose=1, save_best_only=True, save_weights_only=False),
                 TensorBoard(log_dir=os.path.join(LOGS_PATH,'log_'+str(ix)), update_freq='epoch'), schedule] 
                                               
    model.fit_generator(tg.data_generator(),
                        steps_per_epoch=len(train_index)//BATCH_SIZE,
                        epochs=EPOCHS,
                        verbose=2,
                        validation_data=vg.data_generator(),
                        validation_steps=len(test_index)//BATCH_SIZE,
                        callbacks=callbacks)


## Testing the model

In [ ]:
def get_test_data():
    # Load test images' paths.
    arr = []
    for dir_ in ['test/']:
        for y in ['CNV', "DME", "DRUSEN", "NORMAL"]:
            for im in os.listdir(os.path.join(DIR,dir_,y)):
                arr.append(os.path.join(DIR,dir_,y,im))
    
    # Get and Store test images and labels.
    x = np.empty((len(arr),)+SHAPE, dtype=np.float16)
    y = np.empty((len(arr), 4), dtype=np.float16)
    
    for ix, path in tqdm(enumerate(arr)):
        img = np.array(cv2.imread(path))
        img = cv2.resize(img, SHAPE[:2])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = normalize(img)
        img = np.expand_dims(img, axis=2)

        if 'CNV' in path:
            label = [1,0,0,0]
        elif 'DME' in path:
            label = [0,1,0,0]
        elif 'DRUSEN' in path:
            label = [0,0,1,0]
        elif 'NORMAL' in path:
            label = [0,0,0,1]
            
        x[ix] = img
        y[ix] = label
        
    return x, y

In [ ]:
x, y = get_test_data()

In [ ]:
# Threshold predictions
def threshold_arr(array):
    
    # Get all predictions from array
    # Find "max confident score location" in predictions
    # Create numpy zeros array
    # Just make "max confident score location" is 1 in numpy zeros array (other's zero)
    # Store thresholded prediction in new array
    
    new_arr = []
    for ix, val in enumerate(array):
        loc = np.array(val).argmax(axis=0)
        k = list(np.zeros((len(val)), dtype=np.float16))
        k[loc]=1
        new_arr.append(k)
        
    return np.array(new_arr, dtype=np.float16)

## Load Best Model

In [ ]:
# Load all models [3-fold(CV)]
models = []
scores = []
for i in range(3):
    print("** MODEL {} **".format(i))
    model = load_model(os.path.join(MODEL_SAVE_PATH,"heysaw_fold_"+str(i)+".h5"), custom_objects={'f1': f1, 'precision': precision, 'recall': recall})
    score = model.evaluate(x, y, verbose=0)
    print(score)
    scores.append(score[0])
    models.append(model)

In [ ]:
# Choose best scored model according to minimum loss
best_model_index = int(np.array(scores).argmin())
model = models[best_model_index]

## Visualize Results

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label\naccuracy={:0.4f}'.format(accuracy))
    plt.savefig("stuffs/confusion_matrix.png", dpi=150)
    plt.show()

In [ ]:
y_preds = threshold_arr(model.predict(x, verbose=0))

results = precision_recall_fscore_support(y, y_preds ,average='macro')
acc = accuracy_score(y, y_preds)

print("Accuracy: {}, F1_Score: {}, Precision: {}, Recall: {}".format(acc, results[2], results[0], results[1]))
print("\n")
print(classification_report(y, y_preds))
print("\n")
cnf_matrix = confusion_matrix(y.argmax(axis=1), y_preds.argmax(axis=1))

plot_confusion_matrix(cm           = cnf_matrix, 
                      normalize    = False,
                      target_names = ['DME', 'CNV', 'DRUSEN', 'NORMAL'],
                      title        = "Confusion Matrix")

## Inference

In [ ]:
for i in range(5): 
    img = np.array(cv2.imread(DIR+"test/NORMAL/NORMAL-2434258-1.jpeg"))
    img = cv2.resize(img, SHAPE[:2])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = normalize(img)
    img = np.expand_dims(img, axis=2)
    
    start = time.time()
    prediction = model.predict(np.expand_dims(img, axis=0), batch_size=1)
    finish = time.time()
    
    print(threshold_arr(prediction))
    print((finish-start)*1000,"ms")
    print("***")